In [3]:
import os

In [4]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip__dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self)->DataIngestionConfig:
          config=self.config.data_ingestion
          create_directories([config.root_dir])

          data_ingestion_config=DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip__dir=config.unzip_dir
          )

          return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config

    def download_file(self)->str:
        '''Fetch data from url'''
        try:
            dataset_url=self.config.source_URL
            zip_download_dir=self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from{dataset_url} into file {zip_download_dir}")

            file_id=dataset_url.split("/")[-2]
            prefix="https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        unzip_path=self.config.unzip__dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-27 19:47:05,312: INFO: common: yaml:config\config.yaml loaded successfully]
[2024-12-27 19:47:05,316: INFO: common: yaml:params.yaml loaded successfully]
[2024-12-27 19:47:05,318: INFO: common: directory:artifacts created successfully]
[2024-12-27 19:47:05,319: INFO: common: directory:artifacts/data_ingestion created successfully]
[2024-12-27 19:47:05,320: INFO: 986971610: Downloading data fromhttps://drive.google.com/file/d/1uLny0GTs7Yqo6tpc2L2lrgtOslbauxBe/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1uLny0GTs7Yqo6tpc2L2lrgtOslbauxBe
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1uLny0GTs7Yqo6tpc2L2lrgtOslbauxBe&confirm=t&uuid=b849c22f-519d-4544-b984-4341fd80a555
To: d:\AI-detector\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [01:25<00:00, 576kB/s]

[2024-12-27 19:48:36,840: INFO: 986971610: Downloaded data from https://drive.google.com/file/d/1uLny0GTs7Yqo6tpc2L2lrgtOslbauxBe/view?usp=sharing into file artifacts/data_ingestion/data.zip]
